In [1]:
import random
from os.path import join

In [2]:
import emcee
import numpy as np
import numpy.random
from numpy.testing import assert_array_equal

In [3]:
import pint.fermi_toas as fermi
import pint.models
import pint.toa as toa
from pint.mcmc_fitter import MCMCFitter, MCMCFitterBinnedTemplate
from pint.sampler import EmceeSampler
from pint.scripts.event_optimize import marginalize_over_phase, read_gaussfitfile
from pinttestdata import datadir, testdir

In [4]:
%pdb

Automatic pdb calling has been turned ON


In [5]:
def test_sampler():
    r = []
    for i in range(2):
        random.seed(0)
        numpy.random.seed(0)
        s = numpy.random.mtrand.RandomState(0)

        parfile = join(datadir, "PSRJ0030+0451_psrcat.par")
        eventfile = join(
            datadir,
            "J0030+0451_P8_15.0deg_239557517_458611204_ft1weights_GEO_wt.gt.0.4.fits",
        )
        gaussianfile = join(datadir, "templateJ0030.3gauss")
        weightcol = "PSRJ0030+0451"

        minWeight = 0.9
        nwalkers = 10
        nsteps = 1
        nbins = 256
        phs = 0.0

        model = pint.models.get_model(parfile)
        tl = fermi.load_Fermi_TOAs(
            eventfile, weightcolumn=weightcol, minweight=minWeight
        )
        ts = toa.TOAs(toalist=tl)
        # Introduce a small error so that residuals can be calculated
        ts.table["error"] = 1.0
        ts.filename = eventfile
        ts.compute_TDBs()
        ts.compute_posvels(ephem="DE421", planets=False)

        weights = np.asarray([x["weight"] for x in ts.table["flags"]])
        template = read_gaussfitfile(gaussianfile, nbins)
        template /= template.mean()
        print(model)
        sampler = EmceeSampler(nwalkers)
        fitter = MCMCFitterBinnedTemplate(
            ts, model, sampler, template=template, weights=weights, phs=phs
        )
        fitter.sampler.random_state = s

        # phases = fitter.get_event_phases()
        # maxbin, like_start = marginalize_over_phase(phases, template,
        #                                            weights=fitter.weights,
        #                                            minimize=True,
        #                                            showplot=True)
        # fitter.fitvals[-1] = 1.0 - maxbin[0] / float(len(template))

        # fitter.set_priors(fitter, 10)
        pos = fitter.sampler.get_initial_pos(
            fitter.fitkeys,
            fitter.fitvals,
            fitter.fiterrs,
            0.1,
            minMJD=fitter.minMJD,
            maxMJD=fitter.maxMJD,
        )
        # pos = fitter.clip_template_params(pos)
        fitter.sampler.initialize_sampler(fitter.lnposterior, fitter.n_fit_params)
        fitter.sampler.run_mcmc(pos, nsteps)
        # fitter.fit_toas(maxiter=nsteps, pos=None)
        # fitter.set_parameters(fitter.maxpost_fitvals)

        # fitter.phaseogram()
        # samples = sampler.sampler.chain[:, 10:, :].reshape((-1, fitter.n_fit_params))

        # r.append(np.random.randn())
        r.append(sampler.sampler.chain[0])
    assert_array_equal(r[0], r[1])

In [6]:
test_sampler()

INFO: TIMESYS TT [pint.fermi_toas]
INFO: TIMEREF GEOCENTRIC [pint.fermi_toas]
INFO: Building geocentric TOAs, with MJDs in range 54682.840987961186 to 57217.91590582851 [pint.fermi_toas]


INFO: Computing TDB columns. [pint.toa]
INFO: Using EPHEM = DE421 for TDB calculation. [pint.toa]


INFO: Computing PosVels of observatories and Earth, using DE421 [pint.toa]
INFO: Set solar system ephemeris to link:
	https://data.nanograv.org/static/data/ephem/de421.bsp [pint.solar_system_ephemerides]
PSRJ                           J0030+0451
UNITS                                 TDB
DILATEFREQ                              N
DMDATA                                  N
NTOA                                    0
CHI2                                  0.0
RAJ                      0:30:27.43030000
DECJ                     4:51:39.74000000
PMRA                                 -5.3
PMDEC                                -2.0
PX                                    0.0
POSEPOCH           52079.0000000000000000
F0                       205.530699274922 1 1e-07
F1                            -4.2976e-16 1 1e-18
PEPOCH             50984.4000000000000000
PLANET_SHAPIRO                          N
DM                                  4.333
DM1                                   0.0
TZRMJD             56000

100%|██████████| 1/1 [00:00<00:00,  1.16it/s]


INFO: TIMESYS TT [pint.fermi_toas]
INFO: TIMEREF GEOCENTRIC [pint.fermi_toas]
INFO: Building geocentric TOAs, with MJDs in range 54682.840987961186 to 57217.91590582851 [pint.fermi_toas]
INFO: Computing TDB columns. [pint.toa]
INFO: Using EPHEM = DE421 for TDB calculation. [pint.toa]


INFO: Computing PosVels of observatories and Earth, using DE421 [pint.toa]
PSRJ                           J0030+0451
UNITS                                 TDB
DILATEFREQ                              N
DMDATA                                  N
NTOA                                    0
CHI2                                  0.0
RAJ                      0:30:27.43030000
DECJ                     4:51:39.74000000
PMRA                                 -5.3
PMDEC                                -2.0
PX                                    0.0
POSEPOCH           52079.0000000000000000
F0                       205.530699274922 1 1e-07
F1                            -4.2976e-16 1 1e-18
PEPOCH             50984.4000000000000000
PLANET_SHAPIRO                          N
DM                                  4.333
DM1                                   0.0
TZRMJD             56000.0000000000000000
TZRSITE                                 1
TZRFRQ                             1400.0

INFO: Fit Keys:	['F0', 'F1

100%|██████████| 1/1 [00:00<00:00,  1.16it/s]


In [7]:
parfile = join(datadir, "PSRJ0030+0451_psrcat.par")
eventfile = join(datadir,"J0030+0451_P8_15.0deg_239557517_458611204_ft1weights_GEO_wt.gt.0.4.fits",)
gaussianfile = join(datadir, "templateJ0030.3gauss")
weightcol = "PSRJ0030+0451"

In [8]:
minWeight = 0.9
nwalkers = 10
nsteps = 1
nbins = 256
phs = 0.0

In [9]:
test_model = pint.models.get_model(parfile)
test_tl = fermi.load_Fermi_TOAs(
            eventfile, weightcolumn=weightcol, minweight=minWeight
        )
test_ts = toa.TOAs(toalist=test_tl)
# Introduce a small error so that residuals can be calculated
test_ts.table["error"] = 1.0
test_ts.filename = eventfile
test_ts.compute_TDBs()
test_ts.compute_posvels(ephem="DE421", planets=False)

INFO: TIMESYS TT [pint.fermi_toas]
INFO: TIMEREF GEOCENTRIC [pint.fermi_toas]
INFO: Building geocentric TOAs, with MJDs in range 54682.840987961186 to 57217.91590582851 [pint.fermi_toas]
INFO: Computing TDB columns. [pint.toa]
INFO: Using EPHEM = DE421 for TDB calculation. [pint.toa]


INFO: Computing PosVels of observatories and Earth, using DE421 [pint.toa]


In [10]:
test_weights = np.asarray([x["weight"] for x in test_ts.table["flags"]])
test_template = read_gaussfitfile(gaussianfile, nbins)
test_template /= test_template.mean()
test_sampler = EmceeSampler(nwalkers)

In [11]:
fitter = MCMCFitterBinnedTemplate(
    test_ts, test_model, test_sampler, template=test_template, weights=test_weights, phs=phs
    )

INFO: Fit Keys:	['F0', 'F1'] [pint.mcmc_fitter]
INFO: Fit Vals:	[ 2.05530699e+02 -4.29760000e-16] [pint.mcmc_fitter]
